In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import json

In [4]:
base_url = 'https://www.amazon.com'
product_page_url = f"{base_url}/s?i=specialty-aps&srs=17296221011&rh=n%3A17296221011&fs=true&qid=1680239013&ref=sr_pg_2&page="

In [5]:
HEADERS = ({'User-Agent':
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'})

In [6]:
##image
def getImage(div):
    try:
        return div.find('img', {'class': 's-image'}).attrs['src']
    except:
        return ''

## returns https://m.media-amazon.com/images/I/71ok9jCkgVL._AC_UY218_.jpg

In [7]:
## title
def getTitle(div):
    try:
        return div.find('h2').text.strip()
    except:
        return ''

##returns Lessons in Chemistry: A Novel

In [8]:
## prod link
def getProdLink(div):
    return div.find('span', {'data-component-type': 's-product-image'}).find('a').attrs["href"]

## returns /Lessons-in-Chemistry-A-Novel/dp/B09BBK79VB/ref=sr_1_1?qid=1680240999&sr=8-1&srs=17296221011

In [9]:
## author
def getAuthor(div):
    try:
        return div.find('h2').next_sibling()[0].text.replace('by', '').strip()
    except:
        return ''

## returns Bonnie Garmus, Miranda Raison, et al.

In [10]:
## rating
def getRating(div):
    try:
        return div.find('div', {'class': 'a-row a-size-small'}).find('span', {'class': 'a-size-base'}).text
    except:
        return '0'

## returns 4.6

In [11]:
## total reviews
def getReviewCount(div):
    try:
        return div.find('div', {'class': 'a-row a-size-small'}).find('span').next_sibling.attrs["aria-label"]
    except:
        return '0'

## returns 126,027

In [12]:
## publisher
def getPublisher(product_soup):
    try:
        return product_soup.find('div', {'id': 'rpi-attribute-audiobook_details-publisher'}).find(
            'div', {'class': 'rpi-attribute-value'}).text.strip()
    except:
        try:
            return product_soup.find('div', {'id': 'rpi-attribute-book_details-publisher'}).find(
                'div', {'class': 'rpi-attribute-value'}).text.strip()
        except:
            pass
    return ''

## returns Random House Audio

In [13]:
## publish date
def getPublishDate(product_soup):
    try:
        return product_soup.find('div', {'id': 'rpi-attribute-audiobook_details-release-date'}).find(
            'div', {'class': 'rpi-attribute-value'}).text.strip()
    except:
        try:
            return product_soup.find('div', {'id': 'rpi-attribute-book_details-publication_date'}).find(
                'div', {'class': 'a-section a-spacing-none a-text-center rpi-attribute-value'}).text.strip()
        except:
            pass
        return 'Jan 1, 1970'

##returns April 5, 2022

In [14]:
##prices
def getPrices(product_soup):
    try:
        prices_list = product_soup.find_all('li', { 'class': 'swatchElement' })
        prices = {}
        for p in prices_list:
            tokens = p.find('a').text.split()
            type, cost = tokens[0], tokens[-1]
            prices[type] = cost
        return prices
    except:
        return {}


##returns {'Kindle': '$14.99',
##         'Audiobook': '$0.00',
##         'Hardcover': '$17.99',
##         'Paperback': '$27.90',
##        'Spiral-bound': '$37.81'}

In [15]:
# breakdown for ratings
# 1, 2, 3, 4, 5

In [16]:
def getRatingBreakdown(product_soup):
    try:
        return [p.text.strip()[:-1] for p in product_soup.find(
            'table', {'id': 'histogramTable'}
        ).find_all('span', { 'class': 'a-size-base' })[1::2][::-1]]
    except:
        return [0,0,0,0,0]

## return ['1', '1', '6', '20', '72']

In [17]:
## ranking
def getRanking(product_soup):
    ranking_tr = []
    try:
        ranking_tr = product_soup.find(
            'span', {'id': 'audibleProductDetailsSalesRank'}
        ).find('td').find('span').find_all('span')
    except:
        try:
            ranking_tr = product_soup.find(
                'ul', {'class': 'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'}
            ).find('ul').find_all('li')
        except:
            pass
    return [t.text for t in ranking_tr]

##returns ['#5 in Audible Books & Originals (See Top 100 in Audible Books & Originals)',
##         '#1 in Historical Fiction (Books)',
##         '#1 in Historical Fiction (Audible Books & Originals)',
##         '#1 in Humorous Fiction & Satire']


In [18]:
pages = 7

In [19]:
result = []

In [24]:
for p in range(1, pages+1):
    page_url = f"{product_page_url}{p}"
    
    resp = requests.get(page_url, headers=HEADERS)
    soup = BeautifulSoup(resp.content, "html")
    book_items = soup.find_all("div", attrs={"data-component-type":'s-search-result'})
    
    for book in book_items:
        book_details  = {}
        book_details["image"] = getImage(book)
        book_details["title"] = getTitle(book)
        book_details["author"] = getAuthor(book)
        book_details["rating"] = getRating(book)
        book_details["review_count"] = getReviewCount(book)
        try:
            link = getProdLink(book)
            book_details["product_link"] = link
            product_url = f"{base_url}{link}"


            prod_resp = requests.get(product_url, headers=HEADERS)
            product_soup = BeautifulSoup(prod_resp.content, 'html')

            book_details["publisher"] = getPublisher(product_soup)
            book_details["publish_date"] = getPublishDate(product_soup)
            book_details["prices"] = getPrices(product_soup)
            book_details["rating_breakdown"] = getRatingBreakdown(product_soup)
            book_details["ranking"] = getRanking(product_soup)
        except:
            print("couldnt load product page")
            pass
        result.append(book_details)
        print(book_details["title"])

Lessons in Chemistry: A Novel
Tomorrow, and Tomorrow, and Tomorrow: A novel
Demon Copperhead: An Oprah's Book Club Pick
Remarkably Bright Creatures: A Read with Jenna Pick
Reminders of Him: A Novel
Mad Honey: A Novel
I'm Glad My Mom Died
Horse: A Novel
Fairy Tale
Lighter: Let Go of the Past, Connect with the Present, and Expand the Future
Finding Me: An Oprah's Book Club Pick
Carrie Soto Is Back: A Novel
An Immense World: How Animal Senses Reveal the Hidden Realms Around Us
Our Missing Hearts: A Novel
Friends, Lovers, and the Big Terrible Thing: A Memoir
Surrender: 40 Songs, One Story


In [25]:
[r for r in result if r["ranking"]==[]]

[{'image': 'https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/71yNgTMEcpL._AC_UY218_.jpg',
  'title': 'Lessons in Chemistry: A Novel',
  'author': 'Bonnie Garmus  | Apr 5, 2022',
  'rating': '4.6',
  'review_count': '129,546',
  'product_link': '/Lessons-Chemistry-Novel-Bonnie-Garmus/dp/038554734X/ref=sr_1_1?qid=1680739440&sr=8-1&srs=17296221011',
  'publisher': '',
  'publish_date': 'Jan 1, 1970',
  'prices': {},
  'rating_breakdown': [0, 0, 0, 0, 0],
  'ranking': []},
 {'image': 'https://m.media-amazon.com/images/W/IMAGERENDERING_521856-T1/images/I/91VcwLdcNHL._AC_UY218_.jpg',
  'title': 'Tomorrow, and Tomorrow, and Tomorrow: A novel',
  'author': 'Gabrielle Zevin  | Jul 5, 2022',
  'rating': '4.4',
  'review_count': '39,320',
  'product_link': '/Tomorrow-novel-Gabrielle-Zevin/dp/0593321200/ref=sr_1_2?qid=1680739440&sr=8-2&srs=17296221011',
  'publisher': '',
  'publish_date': 'Jan 1, 1970',
  'prices': {},
  'rating_breakdown': [0, 0, 0, 0, 0],
  'ranking': []},


In [22]:
f = open('output-full.json', 'w')

In [23]:
json.dump(result, f)

In [102]:
f.close()